In [115]:
# Load packages
import pandas as pd
import requests
import json
from pathlib import Path

In [116]:
# Define file path and file name
file_path = '../data/raw/'
file_name = 'consumer-price-index_us_1982-84_2022'

In [117]:
# Define series id for API request
seriesid = 'CUUR0000SA0' # CPI for All Urban Consumers (CPI-U) 1982-84=100 (Unadjusted) - CUUR0000SA0

In [118]:
# Request series data from API
headers = {'Content-type': 'application/json'}
data = json.dumps({
    'seriesid': [seriesid],
    'startyear': 2022,
    'endyear': 2022
    })
r = requests.post('https://api.bls.gov/publicAPI/v1/timeseries/data/', headers=headers, data=data)

In [119]:
# Create data frame from json response
df = pd.DataFrame.from_dict(r.json()['Results']['series'][0])

In [120]:
# Split data column into individual columns
df = df.data.apply(pd.Series)

In [121]:
# Create monthly period index
df.period = df.year + df.period
df.period = df.period.str.replace('M', '-')
df.index = pd.DatetimeIndex(df.period).to_period('M')

In [122]:
# Drop superfluous columns
df = df[['value']]

In [123]:
# Sort values by month
df.sort_index(inplace=True)

In [124]:
# Save CPI data
df.to_csv(f'{file_path}{file_name}.csv')